<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/practice/BERT_base_sentence_classification_with_BioASQ_preprocessing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets transformers[sentencepiece]
!pip install rouge

import transformers
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
from datasets import load_dataset
import numpy as np
import csv
from csv import reader
import json
from rouge import Rouge

     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 55.6 MB/s 
     |████████████████████████████████| 636 kB 21.9 MB/s 
     |████████████████████████████████| 895 kB 35.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 6.6 MB/s 
     |████████████████████████████████| 243 kB 58.2 MB/s 
     |████████████████████████████████| 118 kB 73.8 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 


In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForQuestionAnswering.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
csv_file = open('data.csv', 'w')
data_file = open('/content/drive/MyDrive/Thesis/training9b.json', 'r')

data_load = json.load(data_file)
csv_write = csv.writer(csv_file)
csv_write.writerow(["question", "snippet", "rouge-2", "label"])

rouge = Rouge()
  
rows = []
top_n = 5
question_num = 0
for question_set in data_load["questions"]:
  if question_num <= 250:
    sub_rows = []
    question = question_set["body"]
    top = []
    top_id = {}
    snip_num = 0
    chosen = 0
    for snippet in question_set["snippets"]:
        snip_text = snippet["text"]
        rouge_score = rouge.get_scores(question, snip_text)
        rouge_2_f = rouge_score[0]["rouge-2"]["f"]
        top.append(rouge_2_f)
        # top.sort(reverse=True)
        # if check <= 16:
        #   print(top)
        #   check += 1
        #   # print(top_id)
        #   if len(top) == 0:
        #     top.append(rouge_2_f)
        #     top_id[rouge_2_f] = snip_num
        #   elif len(top) < top_n:
        #     top.append(rouge_2_f)
        #     top_id[rouge_2_f] = snip_num
        #   elif len(top) == top_n:
        #     if rouge_2_f >= top[top_n-1]:
        #       top_id.pop(top[top_n-1])
        #       top.pop(top_n-1)
        #       top.append(rouge_2_f)
        #       top_id[rouge_2_f] = snip_num
        sub_rows.append([question, snip_text, rouge_2_f])
        #   snip_num += 1
        #   print(top_id)
      
    top_indx = sorted(range(len(top)), key=lambda i: top[i], reverse=True)[:5]
    # print(top_indx)

    for indx in top_indx:
      sub_rows[indx].append(1)

    for row in sub_rows:
      if len(row) < 4:
        row.append(0)
      # score = row[2]
      # if len(top) < top_n:
      #   row.append(1)
      # elif score >= top[top_n-1] and len(top) == top_n and chosen < top_n:
      #   row.append(1)
      #   chosen += 1
      # else:
      #   row.append(0)


    rows.append(sub_rows)
  question_num += 1

for final_row in rows:
  csv_write.writerows(final_row)

csv_file.close()
data_file.close()

CSV column format is


*   Question
*   Snippet
*   ROUGE score
*   Label




In [4]:
datasets = load_dataset('csv', data_files=['data.csv'])
print(datasets)

Using custom data configuration default-4471b083c09ac07e


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4471b083c09ac07e/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['question', 'snippet', 'rouge-2', 'label'],
        num_rows: 3829
    })
})


In [5]:
def tokenize_dataset(dataset):
  encoded = tokenizer(
      dataset["question"],
      dataset["snippet"],
      padding=True,
      truncation=True,
      return_tensors="np",
  )
  return encoded.data

In [6]:
# tok_data = {
#     split: tokenize_dataset(datasets[split]) for split in datasets.keys()
# }

tok_data = tokenize_dataset(datasets["train"])

In [7]:
# print(tok_data["train"])
# print(datasets["train"].features)
# print()
# print(tok_data["validation"])
# print(datasets["validation"].features)

print(tok_data)

{'input_ids': array([[  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       [  101,  2003, 28127, ...,     0,     0,     0],
       ...,
       [  101,  2515,  5255, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0],
       [  101,  2029, 16012, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}


In [8]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [9]:
bat_size = 8
epoch_num = 8

train_steps = (len(tok_data["input_ids"]) // bat_size) * epoch_num
lr_scheduler = PolynomialDecay(
    initial_learning_rate=1e-4,
    end_learning_rate=0.,
    decay_steps=train_steps
    )

In [10]:
from tensorflow.keras.optimizers import Adam

In [11]:
opt = Adam(learning_rate=lr_scheduler)

In [12]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input

Implementing linear regression into the system

In [13]:
new_model_inputs = Input(shape = (322,), dtype='int64')
new_model = (model)(new_model_inputs)
flat_model = Flatten()(new_model[0])
classifier = Dense(units=len(tok_data["input_ids"]), activation="linear")(flat_model)
fin_model = Model(inputs=new_model_inputs, outputs=classifier)
fin_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [14]:
fin_model.compile(optimizer=opt, loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

fin_model.fit(
    tok_data["input_ids"],
    np.array(datasets["train"]["label"]),
    batch_size=bat_size,
)

479/479 [==============================] - 314s 628ms/step - loss: 0.8090 - accuracy: 0.6451


In [ ]:
predictions = model.predict(tok_data)

In [ ]:
predictions

TFQuestionAnsweringModelOutput([('start_logits',
                                 array([[ 4.769508 ,  4.455809 , -3.9626899, ..., -4.5164957, -4.4873757,
                                         -4.488189 ],
                                        [ 4.769729 ,  4.4800153, -3.8796225, ..., -4.474586 , -4.4288764,
                                         -4.40615  ],
                                        [ 4.7860093,  4.522569 , -3.9724839, ..., -4.5278516, -4.5019965,
                                         -4.4963517],
                                        ...,
                                        [ 4.766457 ,  4.507428 , -3.0532732, ..., -4.676316 , -4.6345677,
                                         -4.63129  ],
                                        [ 4.868543 ,  4.612654 , -3.2444015, ..., -4.5861573, -4.5640416,
                                         -4.5730934],
                                        [ 4.851833 ,  4.6045957, -3.2252214, ..., -4.570398 , -4.5682116,


In [ ]:
highest_predictions = np.argmax(predictions, axis=1)

In [ ]:
from datasets import load_metric

metric = load_metric("glue", "sst2")
metric.compute(predictions=highest_predictions, references=datasets["train"]["label"])